In [1]:
import numpy as np
import cv2
import time
import matplotlib.pyplot as plt

In [2]:
start_index = 4000
img_count = 2000
end_index = start_index + img_count
directory = (
    "/home/michael/Stuff/ffmpeg-tutorial/frames/lviv-2024-06-27/13:50/external/"
)

In [3]:
def split_array(img: np.ndarray, kernel: tuple):
    img_height, img_width = img.shape
    tile_height, tile_width = kernel
    tiled_array = img.reshape(
        img_height // tile_height, tile_height, img_width // tile_width, tile_width
    )
    tiled_array = tiled_array.swapaxes(1, 2)
    return tiled_array

In [4]:
arr1 = np.array(
    [
        [1, 2, 3, 4, 5, 6],
        [7, 8, 9, 10, 11, 12],
        [13, 14, 15, 16, 17, 18],
        [19, 20, 21, 22, 23, 24],
    ]
)
arr2 = np.array(
    [
        [1, 2, 3, 5, 5, 6],
        [7, 8, 10, 11, 11, 12],
        [13, 14, 15, 16, 17, 18],
        [19, 20, 21, 22, 23, 24],
    ]
)
arr1 = split_array(arr1, (2, 2))
arr2 = split_array(arr2, (2, 2))
diff = arr2 - arr1
print(diff)

np.max(np.mean(diff, axis=(2, 3)))
# np.max(np.mean(diff, axis=(1, 2)))
# diff.shape

[[[[0 0]
   [0 0]]

  [[0 1]
   [1 1]]

  [[0 0]
   [0 0]]]


 [[[0 0]
   [0 0]]

  [[0 0]
   [0 0]]

  [[0 0]
   [0 0]]]]


0.75

In [5]:
image = np.array([
    [[255, 0, 0], [0, 255, 0], [0, 0, 255]],
    [[255, 255, 0], [0, 255, 255], [255, 0, 255]],
    [[128, 128, 128], [64, 64, 64], [32, 32, 32]]
])
average_image = np.mean(image, axis=2)

print("Original image array:")
print(image)
print("\nAverage image array (2D):")
print(average_image)
image.shape

Original image array:
[[[255   0   0]
  [  0 255   0]
  [  0   0 255]]

 [[255 255   0]
  [  0 255 255]
  [255   0 255]]

 [[128 128 128]
  [ 64  64  64]
  [ 32  32  32]]]

Average image array (2D):
[[ 85.  85.  85.]
 [170. 170. 170.]
 [128.  64.  32.]]


(3, 3, 3)

In [6]:
prev_img = cv2.imread(f"{directory}/{start_index}.jpg")
prev_img_tiled = split_array(cv2.cvtColor(prev_img, cv2.COLOR_BGR2GRAY), (48, 64))

index = start_index
while True:
    index += 1
    if index == end_index:
        break

    current_img = cv2.imread(f"{directory}/{index}.jpg")

    start_time = time.time()
    current_img_tiled = split_array(
        cv2.cvtColor(current_img, cv2.COLOR_BGR2GRAY), (48, 64)
    )

    diff = np.max(np.mean(current_img_tiled - prev_img_tiled, axis=(2, 3)))
    end_time = time.time()

    print(f"img: {index}, diff {diff}, time: {(end_time - start_time)*1000} ms")
    prev_img = current_img
    cv2.imshow(f"img", current_img)
    # box = cv2.selectROI("ROI", current_img)
    # print(box)
    key = cv2.waitKey(0) & 0xFF

    if key == ord("q"):
        break
    elif key == 81:  # left arrow
        index -= 2
        prev_img = cv2.imread(f"{directory}/{index}.jpg")
        prev_img_tiled = split_array(cv2.cvtColor(prev_img, cv2.COLOR_BGR2GRAY), (48, 64))
    elif key == 83:  # right arrow
        prev_img = current_img
        prev_img_tiled = current_img_tiled

cv2.destroyAllWindows()

img: 4001, diff 186.21907552083334, time: 0.4839897155761719 ms
img: 4002, diff 190.24381510416666, time: 0.45299530029296875 ms
img: 4003, diff 197.55078125, time: 0.4315376281738281 ms
img: 4004, diff 51.197265625, time: 0.43392181396484375 ms
img: 4005, diff 58.7685546875, time: 0.48613548278808594 ms
img: 4004, diff 51.197265625, time: 0.4260540008544922 ms
img: 4003, diff 197.55078125, time: 0.3974437713623047 ms
img: 4004, diff 51.197265625, time: 0.4336833953857422 ms
img: 4003, diff 197.55078125, time: 0.42724609375 ms
img: 4004, diff 51.197265625, time: 0.4887580871582031 ms
img: 4003, diff 197.55078125, time: 0.44035911560058594 ms
img: 4004, diff 51.197265625, time: 0.4222393035888672 ms
img: 4005, diff 61.461588541666664, time: 0.461578369140625 ms
img: 4006, diff 68.4931640625, time: 0.43582916259765625 ms
img: 4007, diff 50.0615234375, time: 0.4630088806152344 ms
img: 4008, diff 124.44173177083333, time: 0.4336833953857422 ms
img: 4009, diff 91.04720052083333, time: 3.324

In [7]:
prev_img = cv2.imread(f"{directory}/{start_index}.jpg")
prev_img = cv2.cvtColor(prev_img, cv2.COLOR_BGR2GRAY)
prev_img = cv2.GaussianBlur(prev_img, (23, 23), 0)
index = start_index
while True:
    index += 1
    if index == end_index:
        break

    current_img = cv2.imread(f"{directory}/{index}.jpg")

    start_time = time.time()
    current_img = cv2.cvtColor(current_img, cv2.COLOR_BGR2GRAY)
    current_img = cv2.GaussianBlur(current_img, (23, 23), 0)
    diff = np.mean(current_img - prev_img)
    end_time = time.time()

    print(f"img: {index}, diff {diff}, time: {(end_time - start_time)*1000} ms")
    prev_img = current_img
    cv2.imshow(f"img", current_img)
    # box = cv2.selectROI("ROI", current_img)
    # print(box)
    key = cv2.waitKey(0) & 0xFF

    if key == ord("q"):
        break
    elif key == 81:  # left arrow
        index -= 2
        prev_img = cv2.imread(f"{directory}/{index}.jpg")
        prev_img = cv2.cvtColor(prev_img, cv2.COLOR_BGR2GRAY)
    elif key == 83:  # right arrow
        prev_prev_img = prev_img
        prev_img = current_img

cv2.destroyAllWindows()

img: 4001, diff 100.601142578125, time: 0.8020401000976562 ms
